In [1]:
import Rust_Machine_Learning
import pandas as pd
from syed_ml_lib import *

In [2]:
# Get the csv data
df = pd.read_csv("../data/iris.data")
# df = pd.read_csv("../data/kmeans-dataset.csv")
# df = pd.read_csv("../data/winequality-red.csv")
# df = pd.read_csv("../data/digit-recognizer/train.csv")
# df = df.sample(n=10000)
df  

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),class
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,Iris-virginica
146,6.3,2.5,5.0,1.9,Iris-virginica
147,6.5,3.0,5.2,2.0,Iris-virginica
148,6.2,3.4,5.4,2.3,Iris-virginica


In [3]:
to_encode = [4] # We won't encode the label column. The output layer will use 0-n (assuming classifications are ordered starting at 0), to determine error.
encoded, feature_to_encoded_cols = one_hot_encode(df, to_encode)
encoded

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),is_ class_Iris-setosa,is_ class_Iris-versicolor,is_ class_Iris-virginica
0,5.1,3.5,1.4,0.2,1.0,0.0,0.0
1,4.9,3.0,1.4,0.2,1.0,0.0,0.0
2,4.7,3.2,1.3,0.2,1.0,0.0,0.0
3,4.6,3.1,1.5,0.2,1.0,0.0,0.0
4,5.0,3.6,1.4,0.2,1.0,0.0,0.0
...,...,...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,0.0,0.0,1.0
146,6.3,2.5,5.0,1.9,0.0,0.0,1.0
147,6.5,3.0,5.2,2.0,0.0,0.0,1.0
148,6.2,3.4,5.4,2.3,0.0,0.0,1.0


In [4]:
# Remove unnecessary data or NAN
to_remove = []
encoded = encoded.drop(encoded.columns[to_remove], axis=1)
encoded

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),is_ class_Iris-setosa,is_ class_Iris-versicolor,is_ class_Iris-virginica
0,5.1,3.5,1.4,0.2,1.0,0.0,0.0
1,4.9,3.0,1.4,0.2,1.0,0.0,0.0
2,4.7,3.2,1.3,0.2,1.0,0.0,0.0
3,4.6,3.1,1.5,0.2,1.0,0.0,0.0
4,5.0,3.6,1.4,0.2,1.0,0.0,0.0
...,...,...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,0.0,0.0,1.0
146,6.3,2.5,5.0,1.9,0.0,0.0,1.0
147,6.5,3.0,5.2,2.0,0.0,0.0,1.0
148,6.2,3.4,5.4,2.3,0.0,0.0,1.0


In [5]:
# Prepare data for training with input rust functions expect
label_col = 2

# Normalize columns
# labels = encoded.iloc[:, label_col]
# max_per_col = encoded.abs().max().replace(0,1)
# encoded = (encoded / max_per_col)
# encoded.iloc[:, label_col] = labels

train_data, test_data = split_train_test(encoded, 0.9, True)

# Remove any encoded data of the same class
train_data_features = train_data.copy()
remove_encoded_category_data_and_label(train_data_features, feature_to_encoded_cols, label_col)

train_data_features_list = train_data_features.values.tolist()

# Data Correlation
corr = train_data_features.corr()
fig = px.imshow(corr, aspect="auto", origin='lower')
fig.show()

# Get labels
train_data_labels = train_data.iloc[:, label_col].values.tolist()
train_data_features

,sepal length (cm),sepal width (cm),petal width (cm),is_ class_Iris-setosa,is_ class_Iris-versicolor,is_ class_Iris-virginica
24,4.8,3.4,0.2,1.0,0.0,0.0
120,6.9,3.2,2.3,0.0,0.0,1.0
27,5.2,3.5,0.2,1.0,0.0,0.0
96,5.7,2.9,1.3,0.0,1.0,0.0
20,5.4,3.4,0.2,1.0,0.0,0.0
...,...,...,...,...,...,...
75,6.6,3.0,1.4,0.0,1.0,0.0
79,5.7,2.6,1.0,0.0,1.0,0.0
42,4.4,3.2,0.2,1.0,0.0,0.0
86,6.7,3.1,1.5,0.0,1.0,0.0


In [6]:
# Now train the model
hidden_layer_sizes = [20,20]
num_classifications = 1 # Tells the library to switch to regression mode
learning_rate = 1e-4
num_iterations = 10000
train_data_features_list
trained_model = Rust_Machine_Learning.BasicNeuralNetwork(train_data_features_list, train_data_labels, hidden_layer_sizes, num_classifications, learning_rate, num_iterations)

% Correct: 0.007407407407407408
% Correct: 0.11851851851851852
% Correct: 0.1925925925925926
% Correct: 0.2518518518518518
% Correct: 0.25925925925925924
% Correct: 0.28888888888888886
% Correct: 0.2962962962962963
% Correct: 0.2962962962962963
% Correct: 0.28888888888888886
% Correct: 0.2962962962962963
% Correct: 0.32592592592592595
% Correct: 0.31851851851851853
% Correct: 0.3333333333333333
% Correct: 0.34814814814814815
% Correct: 0.362962962962963
% Correct: 0.35555555555555557
% Correct: 0.362962962962963
% Correct: 0.37037037037037035
% Correct: 0.37037037037037035
% Correct: 0.3851851851851852
% Correct: 0.37037037037037035
% Correct: 0.37037037037037035
% Correct: 0.37037037037037035
% Correct: 0.37037037037037035
% Correct: 0.37777777777777777
% Correct: 0.37037037037037035
% Correct: 0.34814814814814815
% Correct: 0.34074074074074073
% Correct: 0.34814814814814815
% Correct: 0.34814814814814815
% Correct: 0.34814814814814815
% Correct: 0.34814814814814815
% Correct: 0.34814

In [7]:
# Now let us test the model
test_data_features = test_data.copy()
remove_encoded_category_data_and_label(test_data_features, feature_to_encoded_cols, label_col)
test_data_features_list = test_data_features.values.tolist()
test_data_labels = test_data[test_data.columns[label_col]].values.tolist()

labelled_results = trained_model.regression(test_data_features_list)

num_correct = 0
tolerance = 0.05
for test_label, result_label in zip(test_data_labels, labelled_results):
  num_correct += (abs(test_label-result_label)/(1.0 if test_label == 0 else test_label) <= tolerance)
print("Percent Correct: ", 100.0 * num_correct / len(test_data_labels))
labelled_results

Percent Correct:  40.0


[5.5574276304011745,
 3.7947513317954544,
 4.029995696379928,
 1.6152851851472707,
 4.0397457032515645,
 4.876670217438977,
 5.715546265514958,
 1.6226066881485746,
 1.514304201448694,
 1.2793954705456752,
 1.412270233765639,
 1.2950914605327781,
 4.705028441916235,
 6.36767278967136,
 3.972817211900746]